# Imports

In [1]:
# Imports
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, classification_report

from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from torchvision.datasets.folder import is_image_file

from vit_pytorch import ViT

from pytorch_tabnet.tab_model import TabNetClassifier
# from ft_transformer import FTTransformer
from transformers import BertModel

from AACN_Model import attention_augmented_resnet18, attention_augmented_inceptionv3, attention_augmented_vgg

In [2]:
# Assuming `crops` and directories (`train_dir`, `val_dir`, `test_dir`) are defined
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# Data Preprocessing

In [ ]:
# Define main directories
base_dir = '/Users/izzymohamed/Desktop/Vision For Social Good/Project/Vision-For-Social-Good/DATA'
crop_root = os.path.join(base_dir, 'color')
split_root = os.path.join(base_dir, 'split')

In [ ]:
# Load CSV data
csv_path = '/mnt/data/multimodal_data.csv'
csv_data = pd.read_csv(csv_path)

In [ ]:
# Separate the image paths and labels from the features
csv_image_paths = csv_data['RGB_Image'].values
csv_labels = csv_data['Label'].values
csv_features = csv_data.drop(columns=['Date', 'Tree_ID', 'Orchard_Mapping_Image', 'UAV_Image', 'RGB_Image', 'Multispectral_RGB_Image', 'Multispectral_REG_Image', 'Multispectral_RED_Image', 'Multispectral_NIR_Image', 'Multispectral_GRE_Image', 'Label']).values.astype(np.float32)

In [ ]:
# Define function to remove .DS_Store files
def remove_ds_store(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file == '.DS_Store' or '.DS_Store' in file:
                file_path = os.path.join(root, file)
                print(f"Removing {file_path}")
                os.remove(file_path)

In [ ]:
# Remove .DS_Store files from base directory
remove_ds_store(base_dir)

In [ ]:
# Function to check if a file is an image file
def is_image_file(filename):
    return filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))

In [ ]:
# Function to split data into train, validation, and test sets
def split_data(base_dir, val_split=0.4, test_split=0.1):
    train_files = []
    val_files = []
    test_files = []

    classes = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
    for cls in classes:
        print(f'Processing class: {cls}')
        class_dir = os.path.join(base_dir, cls)

        images = [f for f in os.listdir(class_dir) if is_image_file(os.path.join(class_dir, f))]

        if len(images) == 0:
            print(f"No images found for class {cls}. Skipping...")
            continue

        # Shuffle images to randomize the selection
        random.shuffle(images)

        try:
            train, test = train_test_split(images, test_size=test_split)
            train, val = train_test_split(train, test_size=val_split / (1 - test_split))
        except ValueError as e:
            print(f"Not enough images to split for class {cls}: {e}")
            continue

        train_files.extend([(os.path.join(class_dir, img), cls) for img in train])
        val_files.extend([(os.path.join(class_dir, img), cls) for img in val])
        test_files.extend([(os.path.join(class_dir, img), cls) for img in test])

    return train_files, val_files, test_files, classes

In [ ]:
# Split data
train_files, val_files, test_files, classes = split_data(crop_root)

In [ ]:
# Use the lists of file paths for your dataset loading and transformations
print(f"Train files: {len(train_files)}")
print(f"Validation files: {len(val_files)}")
print(f"Test files: {len(test_files)}")

In [ ]:
# Define the standard image sizes
inception_size = 299
other_size = 224

In [ ]:
# Update the data transformations
data_transforms = {
    'InceptionV3': {
        'train': transforms.Compose([
            transforms.Resize((inception_size, inception_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize((inception_size, inception_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize((inception_size, inception_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    },
    'Others': {
        'train': transforms.Compose([
            transforms.Resize((other_size, other_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize((other_size, other_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.Resize((other_size, other_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
}

In [ ]:
# Class to create the datasets and data loaders and to map between the different modalities
class CustomMultimodalDataset(Dataset):
    def __init__(self, file_paths, csv_features, csv_labels, class_to_idx, transform=None):
        self.file_paths = file_paths
        self.csv_features = csv_features
        self.csv_labels = csv_labels
        self.class_to_idx = class_to_idx
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path, cls = self.file_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.class_to_idx[cls]
        csv_row = self.csv_features[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, csv_row, label

In [ ]:
# Create a mapping from class names to indices
class_to_idx = {cls: idx for idx, cls in enumerate(classes)}

In [ ]:
# Create datasets and data loaders
train_dataset_inception = CustomMultimodalDataset(train_files, csv_features, csv_labels, class_to_idx, transform=data_transforms['InceptionV3']['train'])
val_dataset_inception = CustomMultimodalDataset(val_files, csv_features, csv_labels, class_to_idx, transform=data_transforms['InceptionV3']['val'])
test_dataset_inception = CustomMultimodalDataset(test_files, csv_features, csv_labels, class_to_idx, transform=data_transforms['InceptionV3']['test'])

train_loader_inception = DataLoader(train_dataset_inception, batch_size=32, shuffle=True)
val_loader_inception = DataLoader(val_dataset_inception, batch_size=32, shuffle=True)
test_loader_inception = DataLoader(test_dataset_inception, batch_size=32, shuffle=False)

In [ ]:
# Loaders for other models
train_dataset_others = CustomMultimodalDataset(train_files, csv_features, csv_labels, class_to_idx, transform=data_transforms['Others']['train'])
val_dataset_others = CustomMultimodalDataset(val_files, csv_features, csv_labels, class_to_idx, transform=data_transforms['Others']['val'])
test_dataset_others = CustomMultimodalDataset(test_files, csv_features, csv_labels, class_to_idx, transform=data_transforms['Others']['test'])

train_loader_others = DataLoader(train_dataset_others, batch_size=32, shuffle=True)
val_loader_others = DataLoader(val_dataset_others, batch_size=32, shuffle=True)
test_loader_others = DataLoader(test_dataset_others, batch_size=32, shuffle=False)

# Modalities Fusion

In [ ]:
# Define the CSV feature extractor models
class TabNetCSVFeatureExtractor(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(TabNetCSVFeatureExtractor, self).__init__()
        self.tabnet = TabNetClassifier(
            input_dim=input_dim,
            output_dim=hidden_dim,
            n_d=hidden_dim,
            n_a=hidden_dim,
            n_steps=3,
            gamma=1.3,
            n_independent=2,
            n_shared=2,
            lambda_sparse=1e-3,
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=2e-2),
            scheduler_fn=None,
            scheduler_params=None,
            mask_type='sparsemax'
        )
        self.linear = nn.Linear(hidden_dim, hidden_dim)
    
    def forward(self, x):
        x = x.cpu().numpy()
        predictions = self.tabnet.predict(x)
        feature_output = torch.tensor(predictions, dtype=torch.float32)
        return self.linear(feature_output)

In [ ]:
# Define the Multimodal Transformer (BERT-based) CSV feature extractor
class BERTCSVFeatureExtractor(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(BERTCSVFeatureExtractor, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.fc = nn.Linear(self.bert.config.hidden_size, hidden_dim)

    def forward(self, x):
        output = self.bert(input_ids=x)[1]
        return self.fc(output)

In [ ]:
# Define the MLP CSV feature extractor
class MLP_CSVFeatureExtractor(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(MLP_CSVFeatureExtractor, self).__init__()
        self.extractor = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )

    def forward(self, x):
        return self.extractor(x)

In [ ]:
# Define the Deep MLP CSV feature extractor
class CSVFeatureExtractor(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=10):
        super(CSVFeatureExtractor, self).__init__()
        layers = [nn.Linear(input_dim, hidden_dim), nn.ReLU()]
        for _ in range(num_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
        self.extractor = nn.Sequential(*layers)

    def forward(self, x):
        return self.extractor(x)

In [ ]:
# Define the Convolutional CSV feature extractor
class ConvCSVFeatureExtractor(nn.Module):
    def __init__(self, input_dim, hidden_dim, seq_len, num_layers=2):
        super(ConvCSVFeatureExtractor, self).__init__()
        self.num_layers = num_layers

        layers = [nn.Conv1d(in_channels=1, out_channels=hidden_dim, kernel_size=3, padding=1), nn.ReLU()]
        for _ in range(num_layers - 1):
            layers.append(nn.Conv1d(in_channels=hidden_dim, out_channels=hidden_dim, kernel_size=3, padding=1))
            layers.append(nn.ReLU())
        self.conv_layers = nn.Sequential(*layers)

        conv_output_size = hidden_dim * (seq_len - (num_layers * 2) + 1)

        self.fc = nn.Linear(conv_output_size, hidden_dim)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
# Define the fusion methods for combining image and CSV features
class FusionModel2(nn.Module):
    def __init__(self, model_name, image_feature_extractor, csv_input_dim, csv_hidden_dim, num_classes, fusion_method, csv_model_type='simple', seq_len=None):
        super(FusionModel2, self).__init__()
        self.model_name = model_name
        self.image_feature_extractor = image_feature_extractor
        self.csv_input_dim = csv_input_dim
        self.csv_hidden_dim = csv_hidden_dim
        self.num_classes = num_classes
        self.fusion_method = fusion_method

        self.image_feature_extractor, self.feature_size = self.initialize_image_feature_extractor(model_name, image_feature_extractor)

        if csv_model_type == 'simple':
            self.csv_feature_extractor = CSVFeatureExtractor(self.csv_input_dim, self.csv_hidden_dim, num_layers=25)
        elif csv_model_type == 'deep':
            self.csv_feature_extractor = CSVFeatureExtractor(self.csv_input_dim, self.csv_hidden_dim, num_layers=50)
        elif csv_model_type == 'conv':
            if seq_len is None:
                raise ValueError("seq_len must be provided for the convolutional model")
            self.csv_feature_extractor = ConvCSVFeatureExtractor(self.csv_input_dim, self.csv_hidden_dim, seq_len, num_layers=2)
        else:
            raise ValueError("Unsupported csv_model_type")

        if self.fusion_method == 'late':
            self.fusion_layer = nn.Linear(self.feature_size + self.csv_hidden_dim, self.num_classes)
        elif self.fusion_method == 'intermediate':
            self.intermediate_layer = nn.Linear(self.feature_size, 512)
            self.fusion_layer = nn.Linear(512 + self.csv_hidden_dim, self.num_classes)
        elif self.fusion_method == 'early':
            early_fusion_size = self.feature_size + self.csv_hidden_dim
            self.early_fusion_layer = nn.Linear(early_fusion_size, self.feature_size)
            self.classifier = nn.Linear(self.feature_size, num_classes)
        else:
            raise ValueError("Unsupported fusion method")

    def initialize_image_feature_extractor(self, model_name, image_feature_extractor):
        if model_name == 'InceptionV3':
            feature_size = image_feature_extractor.fc.in_features
            image_feature_extractor.aux_logits = False
            image_feature_extractor.AuxLogits = None
            image_feature_extractor.fc = nn.Identity()
        elif model_name == 'ResNet152' or model_name == 'AttentionAugmentedResNet18':
            feature_size = image_feature_extractor.fc.in_features
            image_feature_extractor.fc = nn.Identity()
        elif model_name == 'VGG19' or model_name == 'AttentionAugmentedVGG19':
            feature_size = image_feature_extractor.classifier[6].in_features
            image_feature_extractor.classifier[6] = nn.Identity()
        elif model_name == 'ViT':
            if hasattr(image_feature_extractor, 'heads'):
                feature_size = image_feature_extractor.heads.head.in_features
                image_feature_extractor.heads.head = nn.Identity()
            elif hasattr(image_feature_extractor, 'classifier'):
                feature_size = image_feature_extractor.classifier.in_features
                image_feature_extractor.classifier = nn.Identity()
            elif hasattr(image_feature_extractor, 'head'):
                feature_size = image_feature_extractor.head.in_features
                image_feature_extractor.head = nn.Identity()
            else:
                for attr_name in dir(image_feature_extractor):
                    attr = getattr(image_feature_extractor, attr_name)
                    if isinstance(attr, nn.Linear):
                        feature_size = attr.in_features
                        setattr(image_feature_extractor, attr_name, nn.Identity())
                        break
                else:
                    raise ValueError(f"Unsupported ViT model structure for model: {model_name}")
        elif model_name == 'AttentionAugmentedInceptionV3':
            inception_model = image_feature_extractor.inception
            feature_size = inception_model.fc.in_features
            inception_model.aux_logits = False
            inception_model.AuxLogits = None
            inception_model.fc = nn.Identity()
        elif model_name == 'AttentionAugmentedVGG19':
            vgg_model = image_feature_extractor.features
            feature_size = vgg_model[-2].out_channels
            vgg_model[-1] = nn.Identity()
        elif model_name == 'AttentionAugmentedResNet18':
            resnet_model = image_feature_extractor
            feature_size = resnet_model.fc.in_features
            resnet_model.fc = nn.Identity()
        else:
            raise ValueError(f"Unsupported model type: {model_name}")

        return image_feature_extractor, feature_size

    def forward(self, img, csv):
        csv_features = self.csv_feature_extractor(csv)

        if self.fusion_method == 'early':
            img = img.view(img.size(0), -1)
            img_csv_combined = torch.cat((img, csv_features), dim=1)
            img_csv_features = self.early_fusion_layer(img_csv_combined)
            output = self.classifier(img_csv_features)
        else:
            img_features = self.image_feature_extractor(img)
            if self.model_name == 'AttentionAugmentedInceptionV3':
                if isinstance(img_features, tuple):
                    img_features = img_features.logits
                    img_features = img_features.view(img_features.size(0), -1)

            if self.fusion_method == 'late':
                combined_features = torch.cat((img_features, csv_features), dim=1)
                output = self.fusion_layer(combined_features)
            elif self.fusion_method == 'intermediate':
                img_features = self.intermediate_layer(img_features)
                combined_features = torch.cat((img_features, csv_features), dim=1)
                output = self.fusion_layer(combined_features)
            else:
                raise ValueError("Unsupported fusion method")
        return output

# Model Training and Evaluation

In [ ]:
# Clear cache function
def clear_cache():
    if torch.backends.mps.is_available():
        torch.mps.empty_cache()
    elif torch.cuda.is_available():
        torch.cuda.empty_cache()
    else:
        torch.cache.empty_cache()

In [ ]:
# Function to adjust learning rate
def adjust_learning_rate(optimizer, epoch, learning_rate):
    lr = learning_rate * (0.1 ** (epoch // 10))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
# Training function with mixed precision and gradient accumulation
def train_model(model, criterion, optimizer, train_loader, val_loader, num_classes, csv_input_dim, device, fusion_method='late', num_epochs=40, initial_lr=0.001):
    early_stopping_patience = 5
    best_val_loss = float('inf')
    patience_counter = 0

    model.to(device)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for i, data in enumerate(train_loader):
            inputs_img, inputs_csv, labels = data
            inputs_img, inputs_csv, labels = inputs_img.to(device), inputs_csv.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs_img, inputs_csv)
            if isinstance(outputs, tuple):
                outputs = outputs[0]

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = 100 * correct / total

        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for data in val_loader:
                inputs_img, inputs_csv, labels = data
                inputs_img, inputs_csv, labels = inputs_img.to(device), inputs_csv.to(device), labels.to(device)
                outputs = model(inputs_img, inputs_csv)
                if isinstance(outputs, tuple):
                    outputs = outputs[0]
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_accuracy = 100 * correct / total

        print(f'Epoch {epoch + 1}/{num_epochs}, '
              f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
              f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                print("Early stopping due to no improvement in validation loss.")
                break

    return model

In [ ]:
# Function to create and train the model
def create_and_train_fusion_model(model, train_loader, val_loader, num_classes, csv_input_dim, device, fusion_method='late', num_epochs=40, initial_lr=0.001):
    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr)
    return train_model(model, criterion, optimizer, train_loader, val_loader, num_classes, csv_input_dim, device, fusion_method=fusion_method, num_epochs=num_epochs, initial_lr=initial_lr)

In [ ]:
# Function to evaluate the fusion model
def evaluate_fusion_model(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs_img, inputs_csv, labels = data
            inputs_img, inputs_csv, labels = inputs_img.to(device), inputs_csv.to(device), labels.to(device)
            outputs = model(inputs_img, inputs_csv)
            if isinstance(outputs, tuple):
                outputs = outputs[0]
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_loss /= len(test_loader)
    test_accuracy = 100 * correct / total
    return test_loss, test_accuracy

In [ ]:
# Train the Model
num_classes_inception = len(class_to_idx)
num_classes_others = len(class_to_idx)

In [ ]:
# Set the number of features in the CSV data
num_heads = 8

In [ ]:
# Define the csv input dimensions
csv_input_dim = csv_features.shape[1]

In [ ]:
# Define the csv hidden dimensions
csv_hidden_dim = 256

In [ ]:
# Define the results dictionary
crop_results = {}

In [ ]:
# Define sequence length for BERT
seq_len = csv_features.shape[0]

In [ ]:
# Iterate over all combinations of fusion methods, CNN models, and CSV feature extractors
for fusion_method in ['intermediate', 'late']:
    print(f'-------------------------- Fusion Method: {fusion_method} --------------------------')
    for csv_model_type in ['simple', 'deep', 'conv']:
        csv_feature_extractors = {
            'bert': lambda: BERTCSVFeatureExtractor(csv_input_dim, csv_hidden_dim),
            'mlp': lambda: MLP_CSVFeatureExtractor(csv_input_dim, csv_hidden_dim),
        }

        cnn_feature_extractors = {
            'InceptionV3': models.inception_v3(pretrained=True).to(device),
            'ResNet152': models.resnet152(pretrained=True).to(device),
            'VGG19': models.vgg19(pretrained=True).to(device),
            'ViT': ViT(
                image_size=224,
                patch_size=16,
                num_classes=num_classes_others,
                dim=256,
                depth=6,
                heads=24,
                mlp_dim=2048,
                dropout=0.1,
                emb_dropout=0.1
            ).to(device),
            "AttentionAugmentedInceptionV3": attention_augmented_inceptionv3(attention=True).to(device),
        }

        if 'InceptionV3' in cnn_feature_extractors:
            cnn_feature_extractors['InceptionV3'].aux_logits = False
        
        print(f'---------------- CSV Model Type: {csv_model_type} ----------------')
        for model_name, image_feature_extractor in cnn_feature_extractors.items():
            image_feature_extractor.to(device)
            print(f'Training {model_name} with {fusion_method} fusion and CSV extractor {csv_model_type}')

            fusion_model = FusionModel2(
                model_name=model_name,
                image_feature_extractor=image_feature_extractor,
                csv_input_dim=csv_input_dim,
                csv_hidden_dim=csv_hidden_dim,
                num_classes=num_classes_others,
                fusion_method=fusion_method,
                csv_model_type=csv_model_type,
                seq_len=seq_len
            ).to(device)

            model = create_and_train_fusion_model(
                fusion_model,
                train_loader_others,
                val_loader_others,
                num_classes_others,
                csv_input_dim,
                device,
                fusion_method,
                initial_lr=0.001
            ).to(device)

            test_loss, test_accuracy = evaluate_fusion_model(model, test_loader_others, nn.CrossEntropyLoss(), device)

            crop_results[f"{model_name}_{fusion_method}_{csv_model_type}"] = {
                'model': model,
                'model_name': model_name,
                'fusion_method': fusion_method,
                'csv_model_type': csv_model_type,
                'test_loss': test_loss,
                'test_accuracy': test_accuracy
            }
            print(f'{model_name} with {fusion_method} fusion and CSV extractor {csv_model_type} Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

            del model
            clear_cache()
            print('\n')
        print('\n')
    print('----------------------------------------------------------------------------------------------------------------------')
    print('\n')

# Display Results

In [ ]:
# Create a results folder
results_base_dir = "/Users/izzymohamed/Desktop/Vision For Social Good/Project/Vision-For-Social-Good/RESULTS/Multimodal"
results_folder = os.path.join(results_base_dir, 'T1')
os.makedirs(results_folder, exist_ok=True)

In [ ]:
# Function to save figures
def save_figure(fig, filename):
    fig.savefig(os.path.join(results_folder, filename))
    plt.close(fig)

In [ ]:
# Plot comparison of accuracy for each model for each crop
def plot_accuracy_comparison(results):
    accuracies = [result['test_accuracy'] for result in results.values()]
    model_names = list(results.keys())

    fig = plt.figure(figsize=(20, 10))
    plt.bar(model_names, accuracies)
    plt.ylabel('Accuracy (%)')
    plt.xlabel('Model')
    plt.show()
    save_figure(fig, 'all_fusion_accuracy_comparison.png')

    if fusion_method == 'late':
        fig = plt.figure(figsize=(20, 10))
        plt.bar(model_names, accuracies)
        plt.ylabel('Accuracy (%)')
        plt.xlabel('Model')
        plt.show()
        save_figure(fig, 'late_accuracy_comparison.png')
    elif fusion_method == 'intermediate':
        fig = plt.figure(figsize=(20, 10))
        plt.bar(model_names, accuracies)
        plt.ylabel('Accuracy (%)')
        plt.xlabel('Model')
        plt.show()
        save_figure(fig, 'intermediate_accuracy_comparison.png')

In [ ]:
# Plot comparison of accuracy for each model for each crop
plot_accuracy_comparison(crop_results)

In [ ]:
# Function to display F1, precision, and recall of all models as a table
def display_model_metrics_table(results, test_loader_inception, test_loader_others):
    metrics_data = []

    for model_name, model_info in results.items():
        if model_name in ['InceptionV3', 'AttentionAugmentedInceptionV3']:
            test_loader = test_loader_inception
        else:
            test_loader = test_loader_others

        model = model_info['model']
        device = next(model.parameters()).device
        model.eval()

        all_labels = []
        all_predicted = []

        for images, csv_features, labels in test_loader:
            images, csv_features, labels = images.to(device), csv_features.to(device), labels.to(device)

            with torch.no_grad():
                outputs = model(images, csv_features)
                _, predicted = torch.max(outputs, 1)

            all_labels.extend(labels.cpu().numpy())
            all_predicted.extend(predicted.cpu().numpy())

        precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predicted, average='macro')

        metrics_data.append({
            'Model': model_name,
            'Precision': precision,
            'Recall': recall,
            'F1-score': f1
        })

    metrics_df = pd.DataFrame(metrics_data)
    display(metrics_df)
    metrics_df.to_csv(os.path.join(results_folder, 'model_metrics.csv'), index=False)

In [ ]:
# Display the table of metrics for all models
display_model_metrics_table(crop_results, test_loader_inception, test_loader_others)

In [ ]:
# Display some correctly and incorrectly classified images
def display_classification_results(model, test_loader, model_name, num_images=5):
    device = next(model.parameters()).device
    model.eval()
    class_labels = list(test_loader.dataset.class_to_idx.keys())
    
    images, labels = next(iter(test_loader))
    images, labels = images[:num_images].to(device), labels[:num_images]
    
    with torch.no_grad():
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
    
    fig, axes = plt.subplots(1, num_images, figsize=(20, 8))
    
    for i in range(num_images):
        ax = axes[i]
        img = images[i].cpu().numpy().transpose((1, 2, 0))
        img = np.clip(img, 0, 1)
        ax.imshow(img)
        ax.set_title(f'True: {class_labels[labels[i]]}\n Pred: {class_labels[predicted[i].cpu()]}')
        ax.axis('off')

    plt.show()
    save_figure(fig, f'{model_name}_classification_results.png')

In [ ]:
# Display results for each crop
for model_name in crop_results.keys():
    if model_name in ['InceptionV3', 'AttentionAugmentedInceptionV3']:
        test_loader = test_loader_inception
    else:
        test_loader = test_loader_others
    
    print(f'Displaying results for {model_name}')
    display_classification_results(crop_results[model_name]['model'], test_loader, model_name)

In [ ]:
# Function to display the classification report of a given model
def display_classification_report(model, test_loader, model_name):
    device = next(model.parameters()).device
    model.eval()

    all_labels = []
    all_predicted = []

    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        with torch.no_grad():
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_predicted.extend(predicted.cpu().numpy())

    report = classification_report(all_labels, all_predicted, target_names=list(test_loader.dataset.class_to_idx.keys()))
    
    print(report)
    
    report_filename = os.path.join(results_folder, f'{model_name}_classification_report.txt')
    with open(report_filename, 'w') as f:
        f.write(report)

In [ ]:
# Display results for each crop
for model_name in crop_results.keys():
    if model_name in ['InceptionV3', 'AttentionAugmentedInceptionV3']:
        test_loader = test_loader_inception
    else:
        test_loader = test_loader_others
        
    print(f'Displaying classification report for {model_name}')
    display_classification_report([model_name]['model'], test_loader, model_name)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
def plot_confusion_matrix(labels, pred_labels, classes, model_name):
    fig = plt.figure(figsize=(50, 50))
    ax = fig.add_subplot(1, 1, 1)
    cm = confusion_matrix(labels, pred_labels)
    cm_display = ConfusionMatrixDisplay(cm, display_labels=classes)
    cm_display.plot(values_format='d', cmap='Blues', ax=ax)
    fig.delaxes(fig.axes[1])
    plt.xticks(rotation=90)
    plt.xlabel('Predicted Label', fontsize=50)
    plt.ylabel('True Label', fontsize=50)

    plt.show()
    save_figure(fig, f'{model_name}_confusion_matrix.png')

In [ ]:
def get_all_labels_and_preds(model, test_loader):
    all_labels = []
    all_preds = []
    device = next(model.parameters()).device
    model.eval()

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    return all_labels, all_preds

In [ ]:
def generate_confusion_matrices(results, test_loader_inception, test_loader_others):
    classes = list(test_loader.dataset.class_to_idx.keys())
    for model_name, model_info in results.items():
        if model_name in ['InceptionV3', 'AttentionAugmentedInceptionV3']:
            test_loader = test_loader_inception
        else:
            test_loader = test_loader_others

        model = model_info['model']
        labels, pred_labels = get_all_labels_and_preds(model, test_loader)
        plot_confusion_matrix(labels, pred_labels, classes, model_name)

In [ ]:
generate_confusion_matrices(crop_results, test_loader_inception, test_loader_others)

In [ ]:
# Function to normalize images
def normalize_image(image):
    image = image - image.min()
    image = image / image.max()
    return image

In [ ]:
# Function to plot the most incorrect predictions
def plot_most_incorrect(incorrect, classes, n_images, model_name, normalize=True):
    rows = int(np.ceil(np.sqrt(n_images)))
    cols = int(np.ceil(n_images / rows))

    fig = plt.figure(figsize=(25, 20))

    for i in range(rows * cols):
        if i >= len(incorrect):
            break
        ax = fig.add_subplot(rows, cols, i + 1)
        image, true_label, probs = incorrect[i]
        image = image.permute(1, 2, 0)
        true_prob = probs[true_label]
        incorrect_prob, incorrect_label = torch.max(probs, dim=0)
        true_class = classes[true_label]
        incorrect_class = classes[incorrect_label]

        if normalize:
            image = normalize_image(image)

        ax.imshow(image.cpu().numpy())
        ax.set_title(f'true label:\n{true_class} ({true_prob:.3f})\n'
                     f'pred label:\n{incorrect_class} ({incorrect_prob:.3f})', fontsize=10)
        ax.axis('off')

    plt.tight_layout()
    fig.subplots_adjust(hspace=0.7)
    
    plt.show()
    save_figure(fig, f'{model_name}_most_incorrect.png')

In [ ]:
def get_all_details(model, test_loader):
    all_labels = []
    all_preds = []
    all_probs = []
    all_images = []
    device = next(model.parameters()).device
    model.eval()

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            probs = F.softmax(outputs, dim=1)

            all_images.extend(images.cpu())
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu())

    return all_images, all_labels, all_preds, all_probs

In [ ]:
N_IMAGES = 36

In [ ]:
def plot_most_incorrect_predictions(results, test_loader_inception, test_loader_others, n_images=36):
    classes = list(test_loader.dataset.class_to_idx.keys())
    for model_name, model_info in results.items():
        if model_name in ['InceptionV3', 'AttentionAugmentedInceptionV3']:
            test_loader = test_loader_inception
        else:
            test_loader = test_loader_others

        model = model_info['model']
        images, labels, pred_labels, probs = get_all_details(model, test_loader)
        corrects = torch.eq(torch.tensor(labels), torch.tensor(pred_labels))
        incorrect_examples = []

        for image, label, prob, correct in zip(images, labels, probs, corrects):
            if not correct:
                incorrect_examples.append((image, label, prob))

    incorrect_examples.sort(key=lambda x: torch.max(x[2], dim=0)[0], reverse=True)
    plot_most_incorrect(incorrect_examples[:n_images], classes, n_images, model_name)

In [ ]:
plot_most_incorrect_predictions(crop_results, test_loader_inception, test_loader_others, N_IMAGES)

In [ ]:
from sklearn import decomposition, manifold

In [ ]:
def get_representations(model, iterator):
    model.eval()
    outputs = []
    labels = []

    with torch.no_grad():
        for x, y in iterator:
            x = x.to(device)
            y_pred = model(x)
            outputs.append(y_pred.cpu())
            labels.append(y)

    outputs = torch.cat(outputs, dim=0)
    labels = torch.cat(labels, dim=0)
    return outputs, labels


In [ ]:
def get_pca(data, n_components=2):
    pca = decomposition.PCA(n_components=n_components)
    pca_data = pca.fit_transform(data)
    return pca_data

In [ ]:
def plot_representations(data, labels, classes, n_images=None):
    if n_images is not None:
        data = data[:n_images]
        labels = labels[:n_images]

    fig = plt.figure(figsize=(15, 15))
    ax = fig.add_subplot(111)
    scatter = ax.scatter(data[:, 0], data[:, 1], c=labels, cmap='hsv')
    plt.show()
    save_figure(fig, f'{model_name}_pca.png')

In [ ]:
outputs, labels = get_representations(model, train_loader)
for model_name in crop_results.keys():
    output_pca_data = get_pca(outputs)
    plot_representations(output_pca_data, labels, classes)

In [ ]:
def get_tsne(data, n_components=2, n_images=None):
    if n_images is not None:
        data = data[:n_images]
    tsne = manifold.TSNE(n_components=n_components, random_state=0)
    tsne_data = tsne.fit_transform(data)
    return tsne_data

In [ ]:
for model_name in crop_results.keys():
    output_tsne_data = get_tsne(outputs)
    plot_representations(output_tsne_data, labels, classes)

In [ ]:
# Function to plot filtered images
def plot_filtered_images(images, filters, model_name, n_filters=None, normalize=True):
    images = torch.cat([i.unsqueeze(0) for i in images], dim=0).cpu()
    filters = filters.cpu()

    if n_filters is not None:
        filters = filters[:n_filters]

    n_images = images.shape[0]
    n_filters = filters.shape[0]

    filtered_images = F.conv2d(images, filters)

    fig = plt.figure(figsize=(30, 30))

    for i in range(n_images):
        image = images[i]
        if normalize:
            image = normalize_image(image)
        ax = fig.add_subplot(n_images, n_filters + 1, i + 1 + (i * n_filters))
        ax.imshow(image.permute(1, 2, 0).numpy())
        ax.set_title('Original')
        ax.axis('off')

        for j in range(n_filters):
            image = filtered_images[i][j]
            if normalize:
                image = normalize_image(image)
            ax = fig.add_subplot(n_images, n_filters + 1, i + 1 + (i * n_filters) + j + 1)
            ax.imshow(image.numpy(), cmap='bone')
            ax.set_title(f'Filter {j + 1}')
            ax.axis('off')

    fig.subplots_adjust(hspace=-0.7)
    plt.show()
    save_figure(fig, f'{model_name}_filtered_images.png')

In [ ]:
N_FILTERS = 7
conv_models = ['ResNet152', 'VGG19', 'InceptionV3', 'AttentionAugmentedInceptionV3']
for model_name, model_info in crop_results.items():
    model = model_info['model']
    if model_name in conv_models:
        if hasattr(model, 'conv1'):
            filters = model.conv1.weight.data
        elif hasattr(model, 'features') and hasattr(model.features, '0'):
            filters = model.features[0].weight.data
        else:
            print(f"Model {model_name} structure is not recognized for convolutional layers.")
            filters = None
    else:
        filters = None

    if filters is not None:
        images = [image for image, label in [train_dataset_others[i] for i in range(N_IMAGES)]]
        plot_filtered_images(images, filters, model_name, n_filters=N_FILTERS)

In [ ]:
def plot_filters(filters, normalize=True):
    filters = filters.cpu()
    n_filters = filters.shape[0]
    rows = int(np.sqrt(n_filters))
    cols = int(np.sqrt(n_filters))

    fig = plt.figure(figsize=(30, 15))

    for i in range(rows * cols):
        image = filters[i]
        if normalize:
            image = normalize_image(image)
        ax = fig.add_subplot(rows, cols, i + 1)
        ax.imshow(image.permute(1, 2, 0))
        ax.axis('off')

    fig.subplots_adjust(wspace=-0.9)
    plt.show()
    save_figure(fig, f'{model_name}_filters.png')

In [ ]:
for model_name, model_info in crop_results.items():
    model = model_info['model']
    if model_name in conv_models:
        if hasattr(model, 'conv1'):
            filters = model.conv1.weight.data
        elif hasattr(model, 'features') and hasattr(model.features, '0'):
            filters = model.features[0].weight.data
        else:
            print(f"Model {model_name} structure is not recognized for convolutional layers.")
            filters = None
    else:
        filters = None

    if filters is not None:
        images = [image for image, label in [train_dataset_others[i] for i in range(N_IMAGES)]]
        plot_filters(filters)

In [ ]:
# Function to generate all results
def generate_all_results(results, test_loader):
    plot_accuracy_comparison(results)
    display_model_metrics_table(results, test_loader)
    generate_confusion_matrices(results, test_loader)
    plot_most_incorrect_predictions(results, test_loader, n_images=36)

    for model_name, model_info in results.items():
        model = model_info['model']
        display_classification_results(model, test_loader, model_name)
        display_classification_report(model, test_loader, model_name)

generate_all_results(crop_results, test_loader_others)